#Range Program
Ghost Okaamii
#
#
This program is for measuring ranges accross different assets and time frames. 

In [17]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_pivots(df):
    """Calculate floor pivot points"""
    pivot = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1)) / 3
    r1 = (2 * pivot) - df['low'].shift(1)
    r2 = pivot + (df['high'].shift(1) - df['low'].shift(1))
    r3 = df['high'].shift(1) + 2*(pivot - df['low'].shift(1))
    s1 = (2 * pivot) - df['high'].shift(1)
    s2 = pivot - (df['high'].shift(1) - df['low'].shift(1))
    s3 = df['low'].shift(1) - 2*(df['high'].shift(1) - pivot)
    
    return pivot, r1, r2, r3, s1, s2, s3

def calculate_ranges(df, periods):
    """Calculate various ranges over different periods"""
    for period in periods:
        # High to Low range
        df[f'{period}dayrange'] = df['high'].rolling(window=period).max() - df['low'].rolling(window=period).min()
        
        # High to Close range
        df[f'{period}day_hi2close'] = df['high'].rolling(window=period).max() - df['close']
        
        # Low to Close range
        df[f'{period}day_lo2close'] = df['close'] - df['low'].rolling(window=period).min()

def get_nq_futures_data(period="2y", interval="1d"):
    """
    Download NQ futures data using yfinance and include pivots, moving averages, and ranges
    """
    try:
        # NQ futures ticker symbol
        ticker = "NQ=F"
        
        # Create ticker object and download with more history
        nq = yf.Ticker(ticker)
        
        # Download data with progress info
        print("Downloading data...")
        df = nq.history(period=period, interval=interval)
        print(f"Downloaded {len(df)} rows of data")
        
        if df.empty:
            print("Error: No data received from yfinance")
            return None
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        
        # Clean up column names
        df.columns = df.columns.str.lower()
        
        # Add settlement price
        df['settlement'] = df['close']
        
        # Calculate and add pivot points
        df['pivot'], df['r1'], df['r2'], df['r3'], df['s1'], df['s2'], df['s3'] = calculate_pivots(df)
        
        # Calculate moving averages
        ma_periods = [5, 10, 20, 50, 100, 200]
        for ma in ma_periods:
            df[f'ma{ma}'] = df['close'].rolling(window=ma).mean()
        
        # Calculate ranges for specified periods
        range_periods = [2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
        calculate_ranges(df, range_periods)
        
        # Create list of columns in desired order
        base_cols = ['date', 'open', 'high', 'low', 'close', 'settlement']
        pivot_cols = ['pivot', 'r1', 'r2', 'r3', 's1', 's2', 's3']
        ma_cols = [f'ma{ma}' for ma in ma_periods]
        range_cols = []
        for period in range_periods:
            range_cols.extend([
                f'{period}dayrange',
                f'{period}day_hi2close',
                f'{period}day_lo2close'
            ])
        
        # Combine all columns in order
        cols = base_cols + pivot_cols + ma_cols + range_cols + ['volume']
        
        # Make sure all required columns exist
        existing_cols = [col for col in cols if col in df.columns]
        df = df[existing_cols]
        
        return df
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

# Get daily data
df = get_nq_futures_data(period="2y")

if df is not None and not df.empty:
    print("\nLast few rows of the data:")
    display(df.tail())
    
    print("\nDataFrame info:")
    display(df.info())
else:
    print("Failed to retrieve data. Please check your internet connection and try again.")


Downloaded 505 rows of data

Last few rows of the data:


,date,open,high,low,close,settlement,pivot,r1,r2,r3,...,100dayrange,100day_hi2close,100day_lo2close,200dayrange,200day_hi2close,200day_lo2close,300dayrange,300day_hi2close,300day_lo2close,volume
500,2024-12-13 00:00:00-05:00,21720.75,21908.00,21666.50,21795.75,21795.75,21676.000000,21747.750000,21844.250000,21916.000000,...,4557.0,112.25,4444.75,4794.75,112.25,4682.5,7767.75,112.25,7655.5,639403
501,2024-12-16 00:00:00-05:00,21780.25,22152.50,21762.00,22110.25,22110.25,21790.083333,21913.666667,22031.583333,22155.166667,...,4801.5,42.25,4759.25,5039.25,42.25,4997.0,8012.25,42.25,7970.0,482433
502,2024-12-17 00:00:00-05:00,22115.00,22124.50,21942.25,22014.75,22014.75,22008.250000,22254.500000,22398.750000,22645.000000,...,4801.5,137.75,4663.75,5039.25,137.75,4901.5,8012.25,137.75,7874.5,277298
503,2024-12-18 00:00:00-05:00,22010.00,22083.75,21034.25,21218.75,21218.75,22027.166667,22112.083333,22209.416667,22294.333333,...,4801.5,933.75,3867.75,5039.25,933.75,4105.5,8012.25,933.75,7078.5,209588
504,2024-12-19 00:00:00-05:00,21151.50,21151.50,20970.25,20974.75,20974.75,21445.583333,21856.916667,22495.083333,22906.416667,...,4801.5,1177.75,3623.75,5039.25,1177.75,3861.5,8012.25,1177.75,6834.5,1512



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 50 columns):
 #   Column           Non-Null Count  Dtype                           
---  ------           --------------  -----                           
 0   date             505 non-null    datetime64[ns, America/New_York]
 1   open             505 non-null    float64                         
 2   high             505 non-null    float64                         
 3   low              505 non-null    float64                         
 4   close            505 non-null    float64                         
 5   settlement       505 non-null    float64                         
 6   pivot            504 non-null    float64                         
 7   r1               504 non-null    float64                         
 8   r2               504 non-null    float64                         
 9   r3               504 non-null    float64                         
 10  s1               504 

None

In [18]:
import tkinter as tk
from tkinter import ttk, messagebox
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import webbrowser
import os

class MarketAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Market Analysis Dashboard")
        
        # Set dark theme
        self.root.configure(bg='black')
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('TFrame', background='black')
        style.configure('TLabel', background='black', foreground='white')
        style.configure('TButton', background='#333333', foreground='white')
        style.configure('TEntry', fieldbackground='#333333', foreground='white')
        
        # Create main frame
        self.main_frame = ttk.Frame(root)
        self.main_frame.pack(padx=20, pady=20, fill='both', expand=True)
        
        # Symbol examples
        self.examples_label = ttk.Label(self.main_frame, text="Example Symbols:", 
                                      font=('Arial', 10, 'bold'))
        self.examples_label.pack(anchor='w', pady=(0,5))
        
        examples_text = """
        Stock: AAPL (Apple Inc.)
        Forex: EURUSD=X (Euro/USD)
        Futures: NQ=F (Nasdaq Futures)
        """
        self.examples = ttk.Label(self.main_frame, text=examples_text)
        self.examples.pack(anchor='w', pady=(0,15))
        
        # Input frame
        self.input_frame = ttk.Frame(self.main_frame)
        self.input_frame.pack(fill='x', pady=(0,15))
        
        self.symbol_label = ttk.Label(self.input_frame, text="Enter Symbol:")
        self.symbol_label.pack(side='left', padx=(0,10))
        
        self.symbol_entry = ttk.Entry(self.input_frame, width=20)
        self.symbol_entry.pack(side='left')
        self.symbol_entry.insert(0, "NQ=F")  # Default symbol
        
        # Generate button
        self.generate_btn = ttk.Button(self.main_frame, text="Generate Dashboard",
                                     command=self.generate_dashboard)
        self.generate_btn.pack(pady=10)
        
    def calculate_support_resistance(self, df, periods=[20]):
        """Calculate support and resistance levels"""
        for period in periods:
            df[f'resistance_{period}d'] = df['high'].rolling(window=period).max()
            df[f'support_{period}d'] = df['low'].rolling(window=period).min()
        return df

    def calculate_pivots(self, df):
        """Calculate floor pivot points"""
        pivot = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1)) / 3
        r1 = (2 * pivot) - df['low'].shift(1)
        r2 = pivot + (df['high'].shift(1) - df['low'].shift(1))
        r3 = df['high'].shift(1) + 2*(pivot - df['low'].shift(1))
        s1 = (2 * pivot) - df['high'].shift(1)
        s2 = pivot - (df['high'].shift(1) - df['low'].shift(1))
        s3 = df['low'].shift(1) - 2*(df['high'].shift(1) - pivot)
        return pivot, r1, r2, r3, s1, s2, s3

    def calculate_ranges(self, df, periods):
        """Calculate various ranges"""
        for period in periods:
            df[f'{period}dayrange'] = df['high'].rolling(window=period).max() - \
                                    df['low'].rolling(window=period).min()
            df[f'{period}day_hi2close'] = df['high'].rolling(window=period).max() - df['close']
            df[f'{period}day_lo2close'] = df['close'] - df['low'].rolling(window=period).min()

    def get_market_data(self, symbol, period="2y", interval="1d"):
        """Get and process market data"""
        try:
            print(f"Downloading data for {symbol}...")
            ticker = yf.Ticker(symbol)
            df = ticker.history(period=period, interval=interval)
            
            if df.empty:
                raise ValueError("No data received from yfinance")
            
            df.reset_index(inplace=True)
            df.columns = df.columns.str.lower()
            df['settlement'] = df['close']
            
            # Calculate indicators
            df = self.calculate_support_resistance(df, periods=[20, 50, 100])
            df['pivot'], df['r1'], df['r2'], df['r3'], df['s1'], df['s2'], df['s3'] = \
                self.calculate_pivots(df)
            
            # Calculate moving averages
            ma_periods = [5, 10, 20, 50, 100, 200]
            for ma in ma_periods:
                df[f'ma{ma}'] = df['close'].rolling(window=ma).mean()
            
            # Calculate ranges
            range_periods = [2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
            self.calculate_ranges(df, range_periods)
            
            return df
            
        except Exception as e:
            raise Exception(f"Error downloading data: {str(e)}")

    def generate_html(self, df, symbol):
        """Generate HTML dashboard"""
        html_content = f'''
        <!DOCTYPE html>
        <html>
        <head>
            <title>{symbol} Analysis Dashboard</title>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/plotly.js/2.24.2/plotly.min.js"></script>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 20px;
                    background-color: #1a1a1a;
                    color: #ffffff;
                }}
                .dashboard {{
                    max-width: 1200px;
                    margin: 0 auto;
                }}
                .chart-container {{
                    background: #2d2d2d;
                    padding: 20px;
                    margin-bottom: 20px;
                    border-radius: 8px;
                }}
                .stats-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 20px;
                    margin-bottom: 20px;
                }}
                .stat-card {{
                    background: #2d2d2d;
                    padding: 15px;
                    border-radius: 8px;
                }}
                .stat-title {{
                    color: #999;
                    font-size: 0.9em;
                    margin-bottom: 5px;
                }}
                .stat-value {{
                    font-size: 1.2em;
                    font-weight: bold;
                    color: #fff;
                }}
            </style>
        </head>
        <body>
            <div class="dashboard">
                <h1>{symbol} Analysis Dashboard</h1>
                
                <div class="stats-grid">
                    <div class="stat-card">
                        <div class="stat-title">Current Price</div>
                        <div class="stat-value">{df['close'].iloc[-1]:.2f}</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-title">Daily Range</div>
                        <div class="stat-value">{df['high'].iloc[-1] - df['low'].iloc[-1]:.2f}</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-title">20D Support</div>
                        <div class="stat-value">{df['support_20d'].iloc[-1]:.2f}</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-title">20D Resistance</div>
                        <div class="stat-value">{df['resistance_20d'].iloc[-1]:.2f}</div>
                    </div>
                </div>

                <div class="chart-container">
                    <div id="price-chart"></div>
                </div>

                <div class="chart-container">
                    <div id="sr-chart"></div>
                </div>

                <div class="chart-container">
                    <div id="range-chart"></div>
                </div>
            </div>

            <script>
                const data = {df.to_json(orient='records', date_format='iso')};
                
                // Price Chart
                const priceTrace = {{
                    type: 'scatter',
                    mode: 'lines',
                    name: 'Price',
                    x: data.map(d => d.date),
                    y: data.map(d => d.close),
                    line: {{color: '#17BECF'}}
                }};

                const ma20Trace = {{
                    type: 'scatter',
                    mode: 'lines',
                    name: 'MA20',
                    x: data.map(d => d.date),
                    y: data.map(d => d.ma20),
                    line: {{color: '#7F7F7F'}}
                }};

                Plotly.newPlot('price-chart', [priceTrace, ma20Trace], {{
                    title: 'Price and MA20',
                    paper_bgcolor: '#2d2d2d',
                    plot_bgcolor: '#2d2d2d',
                    font: {{color: '#fff'}},
                    xaxis: {{gridcolor: '#444', title: 'Date'}},
                    yaxis: {{gridcolor: '#444', title: 'Price'}}
                }});

                // Support/Resistance Chart
                const srChart = {{
                    type: 'scatter',
                    mode: 'lines',
                    name: 'Support/Resistance',
                    x: data.map(d => d.date),
                    y: data.map(d => d.support_20d),
                    line: {{color: '#2CA02C'}}
                }};

                Plotly.newPlot('sr-chart', [srChart], {{
                    title: 'Support Levels',
                    paper_bgcolor: '#2d2d2d',
                    plot_bgcolor: '#2d2d2d',
                    font: {{color: '#fff'}},
                    xaxis: {{gridcolor: '#444', title: 'Date'}},
                    yaxis: {{gridcolor: '#444', title: 'Level'}}
                }});

                // Range Chart
                const rangeChart = {{
                    type: 'scatter',
                    mode: 'lines',
                    name: '20-Day Range',
                    x: data.map(d => d.date),
                    y: data.map(d => d['20dayrange']),
                    line: {{color: '#9467BD'}}
                }};

                Plotly.newPlot('range-chart', [rangeChart], {{
                    title: '20-Day Price Range',
                    paper_bgcolor: '#2d2d2d',
                    plot_bgcolor: '#2d2d2d',
                    font: {{color: '#fff'}},
                    xaxis: {{gridcolor: '#444', title: 'Date'}},
                    yaxis: {{gridcolor: '#444', title: 'Range'}}
                }});
            </script>
        </body>
        </html>
        '''
        
        # Save HTML file
        filename = f"{symbol}_dashboard.html"
        with open(filename, 'w') as f:
            f.write(html_content)
        return filename

    def generate_dashboard(self):
        """Generate and display the dashboard"""
        try:
            symbol = self.symbol_entry.get().strip().upper()
            if not symbol:
                messagebox.showerror("Error", "Please enter a symbol")
                return
            
            # Get data
            df = self.get_market_data(symbol)
            
            # Generate HTML
            filename = self.generate_html(df, symbol)
            
            # Open in default browser
            webbrowser.open('file://' + os.path.realpath(filename))
            
            messagebox.showinfo("Success", 
                              f"Dashboard generated for {symbol}!\nFile: {filename}")
            
        except Exception as e:
            messagebox.showerror("Error", str(e))

def main():
    root = tk.Tk()
    app = MarketAnalysisApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

: 